In [57]:
# muat package atau library yang dibutuhkan
import pandas as pd
from catboost import CatBoostClassifier
from catboost import Pool, cv
from catboost import EFstrType
from catboost import EShapCalcType, EFeaturesSelectionAlgorithm
import feature_engine
from datetime import datetime
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_validate


In [2]:
# muat dataset
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [3]:
# pisahkan antara predictor dan target atau label
# untuk dataset train
X = train_df.drop(columns=['default'])
y = train_df['default']

In [4]:
# pisahkan dataset train menjadi 2
# dataset train dan validation
# kita akan menggunakan hasil StratifiedKFold
# tidak menggunakan train_test_split
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=1)

In [5]:
skf.split(X, y)

<generator object _BaseKFold.split at 0x000001EC45F18B30>

In [6]:
for train_index, val_index in skf.split(X, y):
    print("TRAIN:", train_index, "VALIDATION:", val_index)

TRAIN: [   0    1    2 ... 3689 3690 3692] VALIDATION: [   3    4   14   16   20   21   29   33   34   36   42   43   48   51
   53   55   56   59   60   64   66   68   76   82   83   84   85   86
   93  101  104  107  108  127  135  142  147  150  151  157  166  171
  181  182  184  187  188  194  195  200  205  207  209  215  220  227
  229  232  233  238  240  242  247  250  254  256  257  265  267  271
  275  279  283  287  295  297  308  310  311  330  333  336  337  341
  343  344  348  350  356  358  359  361  364  367  368  369  372  387
  388  391  393  403  411  414  426  428  430  431  434  435  437  438
  441  453  455  459  472  474  477  479  480  483  489  490  491  492
  501  506  507  510  511  512  523  524  525  529  530  532  533  535
  536  548  556  561  565  568  569  572  574  579  584  590  593  599
  603  605  609  610  611  617  622  630  631  644  647  648  650  657
  658  666  668  680  681  685  686  704  707  710  712  719  723  725
  728  731  735  738  

In [7]:
index_dict={}

for split in range(skf.n_splits):
    index_dict[split]={}
    
    for train_index, val_index in skf.split(X, y):
        index_dict[split]['train'] = train_index
        index_dict[split]['val'] = val_index


In [8]:
# menggunakan hasil split pertama, index split 0
X_train, X_val = X.iloc[index_dict[0]['train']], X.iloc[index_dict[0]['val']]
y_train, y_val = y.iloc[index_dict[0]['train']], y.iloc[index_dict[0]['val']]

In [9]:
X_train

,customer_id,customer_bod,gender,phone_flag,student,employment,credit_card,balance,income,tenure
0,8300,1993-08-17,Female,1.0,No,Self Employed,1.0,87104.12,5015120.75,4yrs 4mon
1,672,2007-12-17,Female,1.0,Yes,NaN,0.0,89236.34,2266076.58,4yrs 1mon
3,2975,1999-11-16,Female,1.0,Yes,NaN,0.0,85979.04,2014246.24,1yrs 8mon
4,3883,1977-08-18,Male,1.0,No,Salaried,0.0,48874.77,5445148.31,0yrs 10mon
5,7825,2002-04-18,Male,1.0,Yes,NaN,0.0,0.00,2390347.61,0yrs 0mon
...,...,...,...,...,...,...,...,...,...,...
3686,5952,1989-01-22,Female,1.0,No,Salaried,0.0,125333.99,5204880.49,5yrs 11mon
3689,219,2004-08-13,Female,1.0,Yes,Self Employed,0.0,192978.31,2682965.60,1yrs 6mon
3690,7289,2002-04-20,Male,1.0,Yes,NaN,0.0,141840.01,2172340.05,1yrs 0mon
3691,698,1978-11-13,Male,1.0,No,Salaried,0.0,69477.43,5556566.10,4yrs 9mon


In [10]:
X_val

,customer_id,customer_bod,gender,phone_flag,student,employment,credit_card,balance,income,tenure
2,5670,2000-02-05,Female,1.0,Yes,NaN,0.0,171553.12,1779347.34,0yrs 9mon
6,6676,1995-08-17,NaN,1.0,No,Salaried,1.0,38546.17,5190882.42,1yrs 1mon
7,7991,1984-04-16,Male,1.0,No,Salaried,0.0,0.00,7112035.30,0yrs 0mon
15,4269,1999-05-12,Male,0.0,No,Salaried,0.0,0.00,3808925.47,0yrs 0mon
17,1216,1970-02-24,Male,1.0,No,Self Employed,0.0,153408.88,4401072.83,2yrs 1mon
...,...,...,...,...,...,...,...,...,...,...
3676,2165,1999-11-17,Female,0.0,Yes,NaN,0.0,146656.96,1089617.48,2yrs 9mon
3681,5465,2000-07-10,Female,1.0,Yes,NaN,0.0,78284.72,1456686.85,1yrs 1mon
3683,6820,1989-02-14,Female,0.0,No,Salaried,0.0,77424.15,5318906.92,3yrs 9mon
3687,6997,1989-12-06,Male,1.0,No,Salaried,0.0,91828.07,3491570.07,1yrs 2mon


In [11]:
X_val.shape[0] / X.shape[0]

0.24993230435959923

Jumlah baris pada dataset _validation_ memiliki proporsi 25%, hasil dari jumlah _fold_ atau _split_ sebesar 4

In [12]:
y_train.value_counts()

0    2579
1     191
Name: default, dtype: int64

In [13]:
y_val.value_counts()

0    859
1     64
Name: default, dtype: int64

In [14]:
# ektraksi dan rekayasa fitur data TRAIN
# kolom 'customer_bod'
# rubah tipe data 'customer_bod' object ke datetime
# dan jadikan kolom baru 'customer_bod_dt',
# ektrak nilai dari tahun, bulan, dan tanggal
# dan masukkan kolom baru 'customer_bod_year',
# 'customer_bod_month', dan 'customer_bod_day'

X_train['customer_bod_dt'] = pd.to_datetime(X_train['customer_bod'])

X_train['customer_bod_year'] = X_train['customer_bod_dt'].dt.year
X_train['customer_bod_month'] = X_train['customer_bod_dt'].dt.month
X_train['customer_bod_day'] = X_train['customer_bod_dt'].dt.day

C:\Users\ASUS\AppData\Local\Temp/ipykernel_25592/4194902132.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['customer_bod_dt'] = pd.to_datetime(X_train['customer_bod'])
C:\Users\ASUS\AppData\Local\Temp/ipykernel_25592/4194902132.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['customer_bod_year'] = X_train['customer_bod_dt'].dt.year
C:\Users\ASUS\AppData\Local\Temp/ipykernel_25592/4194902132.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [15]:
# buat kolom baru berisi nilai usia customer
# usia = hari ini - bod

# today = datetime.today()
today = datetime(2021, 10, 11)

X_train['age_in_days'] = (today - X_train['customer_bod_dt']).dt.days

C:\Users\ASUS\AppData\Local\Temp/ipykernel_25592/3542357448.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['age_in_days'] = (today - X_train['customer_bod_dt']).dt.days


In [16]:
# ektraksi dan rekayasa fitur
# kolom 'tenure'
# membuat kolom baru hasil ektraksi
# berupa nilai tenure dalam bulan denga tipe data numerik


def tenure_in_months(series):
    tenure_splitted = series.str.split(expand=True)
    tenure_calculated = tenure_splitted[0].str.strip("yrs").astype(
        "int"
    ) * 12 + tenure_splitted[1].str.strip("mon").astype("int")
    return tenure_calculated


X_train["tenure_in_months"] = tenure_in_months(X_train["tenure"])


C:\Users\ASUS\AppData\Local\Temp/ipykernel_25592/2122811459.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["tenure_in_months"] = tenure_in_months(X_train["tenure"])


In [17]:
# ektraksi dan rekayasa fitur data VALIDATION
# kolom 'customer_bod'
# rubah tipe data 'customer_bod' object ke datetime
# dan jadikan kolom baru 'customer_bod_dt',
# ektrak nilai dari tahun, bulan, dan tanggal
# dan masukkan kolom baru 'customer_bod_year',
# 'customer_bod_month', dan 'customer_bod_day'

X_val['customer_bod_dt'] = pd.to_datetime(X_val['customer_bod'])

X_val['customer_bod_year'] = X_val['customer_bod_dt'].dt.year
X_val['customer_bod_month'] = X_val['customer_bod_dt'].dt.month
X_val['customer_bod_day'] = X_val['customer_bod_dt'].dt.day

C:\Users\ASUS\AppData\Local\Temp/ipykernel_25592/2445929881.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val['customer_bod_dt'] = pd.to_datetime(X_val['customer_bod'])
C:\Users\ASUS\AppData\Local\Temp/ipykernel_25592/2445929881.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val['customer_bod_year'] = X_val['customer_bod_dt'].dt.year
C:\Users\ASUS\AppData\Local\Temp/ipykernel_25592/2445929881.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [18]:
# buat kolom baru berisi nilai usia customer
# usia = hari ini - bod

# today = datetime.today()
today = datetime(2021, 10, 11)

X_val['age_in_days'] = (today - X_val['customer_bod_dt']).dt.days

C:\Users\ASUS\AppData\Local\Temp/ipykernel_25592/501705899.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val['age_in_days'] = (today - X_val['customer_bod_dt']).dt.days


In [19]:
# ektraksi dan rekayasa fitur
# kolom 'tenure'
# membuat kolom baru hasil ektraksi
# berupa nilai tenure dalam bulan denga tipe data numerik

X_val["tenure_in_months"] = tenure_in_months(X_val["tenure"])

C:\Users\ASUS\AppData\Local\Temp/ipykernel_25592/2523491973.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val["tenure_in_months"] = tenure_in_months(X_val["tenure"])


In [20]:
# ektraksi dan rekayasa fitur data TEST
# kolom 'customer_bod'
# rubah tipe data 'customer_bod' object ke datetime
# dan jadikan kolom baru 'customer_bod_dt',
# ektrak nilai dari tahun, bulan, dan tanggal
# dan masukkan kolom baru 'customer_bod_year',
# 'customer_bod_month', dan 'customer_bod_day'

test_df['customer_bod_dt'] = pd.to_datetime(test_df['customer_bod'])

test_df['customer_bod_year'] = test_df['customer_bod_dt'].dt.year
test_df['customer_bod_month'] = test_df['customer_bod_dt'].dt.month
test_df['customer_bod_day'] = test_df['customer_bod_dt'].dt.day

In [21]:
# buat kolom baru berisi nilai usia customer
# usia = hari ini - bod

# today = datetime.today()
today = datetime(2021, 10, 11)

test_df['age_in_days'] = (today - test_df['customer_bod_dt']).dt.days

In [22]:
# ektraksi dan rekayasa fitur
# kolom 'tenure'
# membuat kolom baru hasil ektraksi
# berupa nilai tenure dalam bulan denga tipe data numerik

test_df["tenure_in_months"] = tenure_in_months(test_df["tenure"])

In [23]:
# imputasi missing value
# seluruh kolom yang memiliki missing value
# merupakan kolom kategori
# sehingga missing value kita asumsikan 
# sebagai kategori baru pada kolom/fitur tersebut

X_train.fillna(value={'gender': 'Missing', 'employment': 'Missing', 'credit_card': -1}, inplace=True)
X_val.fillna(value={'gender': 'Missing', 'employment': 'Missing', 'credit_card': -1}, inplace=True)
test_df.fillna(value={'gender': 'Missing', 'employment': 'Missing', 'credit_card': -1}, inplace=True)

C:\Users\ASUS\anaconda3\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [24]:
X_train.isnull().sum()

customer_id           0
customer_bod          0
gender                0
phone_flag            0
student               0
employment            0
credit_card           0
balance               0
income                0
tenure                0
customer_bod_dt       0
customer_bod_year     0
customer_bod_month    0
customer_bod_day      0
age_in_days           0
tenure_in_months      0
dtype: int64

In [25]:
X_val.isnull().sum()

customer_id           0
customer_bod          0
gender                0
phone_flag            0
student               0
employment            0
credit_card           0
balance               0
income                0
tenure                0
customer_bod_dt       0
customer_bod_year     0
customer_bod_month    0
customer_bod_day      0
age_in_days           0
tenure_in_months      0
dtype: int64

In [26]:
test_df.isnull().sum()

customer_id           0
customer_bod          0
gender                0
phone_flag            0
student               0
employment            0
credit_card           0
balance               0
income                0
tenure                0
customer_bod_dt       0
customer_bod_year     0
customer_bod_month    0
customer_bod_day      0
age_in_days           0
tenure_in_months      0
dtype: int64

In [27]:
X_train

,customer_id,customer_bod,gender,phone_flag,student,employment,credit_card,balance,income,tenure,customer_bod_dt,customer_bod_year,customer_bod_month,customer_bod_day,age_in_days,tenure_in_months
0,8300,1993-08-17,Female,1.0,No,Self Employed,1.0,87104.12,5015120.75,4yrs 4mon,1993-08-17,1993,8,17,10282,52
1,672,2007-12-17,Female,1.0,Yes,Missing,0.0,89236.34,2266076.58,4yrs 1mon,2007-12-17,2007,12,17,5047,49
3,2975,1999-11-16,Female,1.0,Yes,Missing,0.0,85979.04,2014246.24,1yrs 8mon,1999-11-16,1999,11,16,8000,20
4,3883,1977-08-18,Male,1.0,No,Salaried,0.0,48874.77,5445148.31,0yrs 10mon,1977-08-18,1977,8,18,16125,10
5,7825,2002-04-18,Male,1.0,Yes,Missing,0.0,0.00,2390347.61,0yrs 0mon,2002-04-18,2002,4,18,7116,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3686,5952,1989-01-22,Female,1.0,No,Salaried,0.0,125333.99,5204880.49,5yrs 11mon,1989-01-22,1989,1,22,11950,71
3689,219,2004-08-13,Female,1.0,Yes,Self Employed,0.0,192978.31,2682965.60,1yrs 6mon,2004-08-13,2004,8,13,6268,18
3690,7289,2002-04-20,Male,1.0,Yes,Missing,0.0,141840.01,2172340.05,1yrs 0mon,2002-04-20,2002,4,20,7114,12
3691,698,1978-11-13,Male,1.0,No,Salaried,0.0,69477.43,5556566.10,4yrs 9mon,1978-11-13,1978,11,13,15673,57


In [28]:
X_val

,customer_id,customer_bod,gender,phone_flag,student,employment,credit_card,balance,income,tenure,customer_bod_dt,customer_bod_year,customer_bod_month,customer_bod_day,age_in_days,tenure_in_months
2,5670,2000-02-05,Female,1.0,Yes,Missing,0.0,171553.12,1779347.34,0yrs 9mon,2000-02-05,2000,2,5,7919,9
6,6676,1995-08-17,Missing,1.0,No,Salaried,1.0,38546.17,5190882.42,1yrs 1mon,1995-08-17,1995,8,17,9552,13
7,7991,1984-04-16,Male,1.0,No,Salaried,0.0,0.00,7112035.30,0yrs 0mon,1984-04-16,1984,4,16,13692,0
15,4269,1999-05-12,Male,0.0,No,Salaried,0.0,0.00,3808925.47,0yrs 0mon,1999-05-12,1999,5,12,8188,0
17,1216,1970-02-24,Male,1.0,No,Self Employed,0.0,153408.88,4401072.83,2yrs 1mon,1970-02-24,1970,2,24,18857,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3676,2165,1999-11-17,Female,0.0,Yes,Missing,0.0,146656.96,1089617.48,2yrs 9mon,1999-11-17,1999,11,17,7999,33
3681,5465,2000-07-10,Female,1.0,Yes,Missing,0.0,78284.72,1456686.85,1yrs 1mon,2000-07-10,2000,7,10,7763,13
3683,6820,1989-02-14,Female,0.0,No,Salaried,0.0,77424.15,5318906.92,3yrs 9mon,1989-02-14,1989,2,14,11927,45
3687,6997,1989-12-06,Male,1.0,No,Salaried,0.0,91828.07,3491570.07,1yrs 2mon,1989-12-06,1989,12,6,11632,14


In [29]:
# drop kolom yang tidak diikutkan untuk train dan test
# pada ketiganya, data train, validation ataupun data test

X_train_c = X_train.drop(
    columns=[
        "customer_id",
        "customer_bod",
        "tenure",
        "customer_bod_dt",
        "customer_bod_year",
        "customer_bod_month",
        "customer_bod_day"
    ]
)

X_val_c = X_val.drop(
    columns=[
        "customer_id",
        "customer_bod",
        "tenure",
        "customer_bod_dt",
        "customer_bod_year",
        "customer_bod_month",
        "customer_bod_day"
    ]
)

test_df_c = test_df.drop(
    columns=[
        "customer_id",
        "customer_bod",
        "tenure",
        "customer_bod_dt",
        "customer_bod_year",
        "customer_bod_month",
        "customer_bod_day"
    ]
)


In [30]:
X_train_c.columns

Index(['gender', 'phone_flag', 'student', 'employment', 'credit_card',
       'balance', 'income', 'age_in_days', 'tenure_in_months'],
      dtype='object')

In [31]:
X_val_c.columns

Index(['gender', 'phone_flag', 'student', 'employment', 'credit_card',
       'balance', 'income', 'age_in_days', 'tenure_in_months'],
      dtype='object')

In [32]:
# Kaerena kita akan menggunakan algoritma
# dan package CatBoost
# kolom yang yang diasumsikan sebagai kategori
# namun berisi numerik kita rubah ke integer

X_train_c_dtype = X_train_c.convert_dtypes()
X_val_c_dtype = X_val_c.convert_dtypes()
test_df_c_dtype = test_df_c.convert_dtypes()

In [36]:
X_train_c_dtype.columns[[0, 1, 2, 3, 4]]

Index(['gender', 'phone_flag', 'student', 'employment', 'credit_card'], dtype='object')

In [34]:
train_pool = Pool(
    data=X_train_c_dtype.values,
    label=y_train.values,
    cat_features=[0, 1, 2, 3, 4],
    feature_names=[f for f in X_train_c_dtype.columns]
)

In [37]:
val_pool = Pool(
    data=X_val_c_dtype.values,
    label=y_val.values,
    cat_features=[0, 1, 2, 3, 4],
    feature_names=[f for f in X_train_c_dtype.columns]
)

In [38]:
# menyiapkan parameter untuk classifier
params = {'loss_function':'Logloss', # objective function
          'eval_metric':'F1', # metric
          'verbose': 200, # output to stdout info about training process every 200 iterations
          'random_seed': 1
         }

In [39]:
# menyiapkan CatBoost classifier
cbc_1 = CatBoostClassifier(**params)

In [40]:
cbc_1.fit(train_pool, eval_set=val_pool, use_best_model=True, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.040757
0:	learn: 0.5893417	test: 0.5490196	best: 0.5490196 (0)	total: 250ms	remaining: 4m 9s
200:	learn: 0.7926829	test: 0.6285714	best: 0.6603774 (168)	total: 13.2s	remaining: 52.6s
400:	learn: 0.8876081	test: 0.6296296	best: 0.6603774 (168)	total: 25.9s	remaining: 38.7s
600:	learn: 0.9418283	test: 0.6607143	best: 0.6727273 (436)	total: 39.2s	remaining: 26s
800:	learn: 0.9703504	test: 0.6548673	best: 0.6727273 (436)	total: 52.2s	remaining: 13s
999:	learn: 0.9973753	test: 0.6491228	best: 0.6727273 (436)	total: 1m 7s	remaining: 0us

bestTest = 0.6727272727
bestIteration = 436

Shrink model to first 437 iterations.


In [42]:
cbc_1.get_feature_importance(prettified=True)

,Feature Id,Importances
0,balance,37.833310
1,age_in_days,15.993024
2,tenure_in_months,11.136442
3,employment,10.870409
4,income,10.833126
5,credit_card,7.500211
6,gender,2.531549
7,phone_flag,1.704814
8,student,1.597116


In [58]:
cbc_1.get_feature_importance(prettified=True, type=EFstrType.ShapValues, data=train_pool)

,0,1,2,3,4,5,6,7,8,9
0,-0.041851,-0.013402,0.042353,1.973124,0.842857,-1.758684,0.518332,0.112655,-0.518752,-5.696387
1,-0.016384,0.007164,-0.064786,-0.144056,-0.085503,-1.887052,-0.826187,1.897821,-0.756515,-5.696387
2,-0.015812,0.023045,-0.088284,-0.176462,-0.110164,-1.608031,-1.101697,-0.350967,0.960312,-5.696387
3,-0.026387,0.041751,0.028237,-0.257810,-0.024233,-1.663968,0.924080,-0.783174,0.287381,-5.696387
4,-0.028404,0.005949,-0.069819,-0.181837,-0.015186,-2.030805,-0.754378,0.103996,-0.175507,-5.696387
...,...,...,...,...,...,...,...,...,...,...
2765,-0.040318,0.027472,0.054941,-0.330397,-0.149909,0.104992,0.693431,-0.883076,-0.693377,-5.696387
2766,0.017233,0.062339,-0.169853,1.016644,-0.264685,3.239351,-0.640039,1.358123,0.207649,-5.696387
2767,-0.021934,0.031322,-0.175162,-0.222146,-0.093293,1.396198,-0.720219,0.152372,0.137265,-5.696387
2768,-0.020061,0.007570,0.011361,-0.327119,-0.076660,-1.592115,0.923496,-0.504875,-0.535573,-5.696387


In [56]:
cbc_1.calc_feature_statistics(train_pool, feature=7, plot=True);

In [68]:
feature_selection_sum = cbc_1.select_features(train_pool, 
                      eval_set=val_pool, 
                      features_for_select='0-8', 
                      num_features_to_select=5, 
                      steps=4,
                      algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
                      shap_calc_type=EShapCalcType.Exact,
                      train_final_model=False,
                      plot=True);

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.040757
Step #1 out of 4
0:	learn: 0.4968553	test: 0.4807692	best: 0.4807692 (0)	total: 101ms	remaining: 1m 41s
200:	learn: 0.7914110	test: 0.6481481	best: 0.6603774 (152)	total: 17.4s	remaining: 1m 9s
400:	learn: 0.8888889	test: 0.6785714	best: 0.6902655 (364)	total: 31.2s	remaining: 46.6s
600:	learn: 0.9621622	test: 0.6785714	best: 0.6902655 (364)	total: 45.5s	remaining: 30.2s
800:	learn: 0.9840426	test: 0.6545455	best: 0.6902655 (364)	total: 59.1s	remaining: 14.7s
999:	learn: 0.9920844	test: 0.6605505	best: 0.6902655 (364)	total: 1m 12s	remaining: 0us

bestTest = 0.6902654867
bestIteration = 364

Shrink model to first 365 iterations.
Feature #8 eliminated
Step #2 out of 4
0:	learn: 0.4296875	test: 0.3855422	best: 0.3855422 (0)	total: 71.8ms	remaining: 1m 11s
200:	learn: 0.7734139	test: 0.6666667	best: 0.6666667 (179)	total: 13.7s	remaining: 54.3s
400:	learn: 0.8621701	test: 0.6972477	best: 0.7272727 (356)	total: 28.3s	remaining: 42.3s
600:	learn: 0.9157303	test

In [69]:
feature_selection_sum

{'selected_features': [2, 3, 5, 6, 7],
 'eliminated_features_names': ['tenure_in_months',
  'phone_flag',
  'gender',
  'credit_card'],
 'loss_graph': {'main_indices': [0, 1, 2, 3],
  'removed_features_count': [0, 1, 2, 3, 4],
  'loss_values': [0.09951630658117286,
   0.09985619822774411,
   0.09778307261848701,
   0.09668064510304679,
   0.09728869016248932]},
 'eliminated_features': [8, 1, 0, 4],
 'selected_features_names': ['student',
  'employment',
  'balance',
  'income',
  'age_in_days']}

In [65]:
feature_selection_sum2 = cbc_1.select_features(train_pool, 
                      eval_set=val_pool, 
                      features_for_select='0-8', 
                      num_features_to_select=5, 
                      steps=4,
                      algorithm=EFeaturesSelectionAlgorithm.RecursiveByPredictionValuesChange,
                      train_final_model=False,
                      plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.040757
Step #1 out of 4
0:	learn: 0.4968553	test: 0.4807692	best: 0.4807692 (0)	total: 166ms	remaining: 2m 45s
200:	learn: 0.7914110	test: 0.6481481	best: 0.6603774 (152)	total: 18.4s	remaining: 1m 13s
400:	learn: 0.8888889	test: 0.6785714	best: 0.6902655 (364)	total: 46.7s	remaining: 1m 9s
600:	learn: 0.9621622	test: 0.6785714	best: 0.6902655 (364)	total: 1m 4s	remaining: 42.8s
800:	learn: 0.9840426	test: 0.6545455	best: 0.6902655 (364)	total: 1m 19s	remaining: 19.7s
999:	learn: 0.9920844	test: 0.6605505	best: 0.6902655 (364)	total: 1m 39s	remaining: 0us

bestTest = 0.6902654867
bestIteration = 364

Shrink model to first 365 iterations.
Feature #1 eliminated
Step #2 out of 4
0:	learn: 0.5752688	test: 0.5245902	best: 0.5245902 (0)	total: 97.7ms	remaining: 1m 37s
200:	learn: 0.7828746	test: 0.6538462	best: 0.6666667 (174)	total: 18.4s	remaining: 1m 13s
400:	learn: 0.8746356	test: 0.6542056	best: 0.6666667 (174)	total: 38.7s	remaining: 57.8s
600:	learn: 0.9508197	t

In [66]:
feature_selection_sum2

{'selected_features': [3, 5, 6, 7, 8],
 'eliminated_features_names': ['phone_flag',
  'student',
  'gender',
  'credit_card'],
 'loss_graph': {'main_indices': [0, 1, 2, 3],
  'removed_features_count': [0, 1, 2, 3],
  'loss_values': [0.09951630658117286,
   0.10160654526234843,
   0.10578631111188715,
   0.10902511493897825]},
 'eliminated_features': [1, 2, 0, 4],
 'selected_features_names': ['employment',
  'balance',
  'income',
  'age_in_days',
  'tenure_in_months']}

In [70]:
cbc_1.get_all_params()

{'nan_mode': 'Min',
 'eval_metric': 'F1',
 'combinations_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=Uniform:TargetBorderCount=1:TargetBorderType=MinEntropy:Prior=0/1:Prior=0.5/1:Prior=1/1',
  'Counter:CtrBorderCount=15:CtrBorderType=Uniform:Prior=0/1'],
 'iterations': 1000,
 'sampling_frequency': 'PerTree',
 'fold_permutation_block': 0,
 'leaf_estimation_method': 'Newton',
 'counter_calc_method': 'SkipTest',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'ctr_leaf_count_limit': 18446744073709551615,
 'bayesian_matrix_reg': 0.10000000149011612,
 'one_hot_max_size': 2,
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'rsm': 1,
 'boost_from_average': False,
 'max_ctr_complexity': 4,
 'model_size_reg': 0.5,
 'simple_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=Uniform:TargetBorderCount=1:TargetBorderType=MinEntropy:Prior=0/1:Prior=0.5/1:Prior=1/1',
  'Counter:CtrBorderCoun

In [74]:
# menyiapkan parameter untuk classifier
params = {'loss_function':'Logloss', # objective function
          'eval_metric':'F1', # metric
          'verbose': 200, # output to stdout info about training process every 200 iterations
          'random_seed': 1
         }

cbc_2 = CatBoostClassifier(**params)

grid = {
    'depth': [3, 4, 5, 6, 7, 8, 9]
}

grid_search_result = cbc_2.grid_search(grid,
                                       train_pool,
                                       cv=3,
                                       stratified=True,
                                       shuffle=True,
                                       refit=True,
                                       plot=True);

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 34.8ms	remaining: 34.8s
200:	learn: 0.6823529	test: 0.6333333	best: 0.6440678 (116)	total: 6.76s	remaining: 26.9s
400:	learn: 0.7272727	test: 0.6440678	best: 0.6440678 (116)	total: 12.7s	remaining: 19s
600:	learn: 0.7490637	test: 0.6440678	best: 0.6440678 (116)	total: 18.7s	remaining: 12.4s
800:	learn: 0.8043478	test: 0.6666667	best: 0.6666667 (663)	total: 24.7s	remaining: 6.14s
999:	learn: 0.8268551	test: 0.6666667	best: 0.6666667 (663)	total: 31.2s	remaining: 0us

bestTest = 0.6666666667
bestIteration = 663

0:	loss: 0.6666667	best: 0.6666667 (0)	total: 31.3s	remaining: 3m 7s
0:	learn: 0.1052632	test: 0.1025641	best: 0.1025641 (0)	total: 31.2ms	remaining: 31.2s
200:	learn: 0.7022901	test: 0.6440678	best: 0.6440678 (160)	total: 10.2s	remaining: 40.6s
400:	learn: 0.7565543	test: 0.6779661	best: 0.6779661 (274)	total: 18.7s	remaining: 27.9s
600:	learn: 0.8158845	test: 0.6666667	best: 0.6779661 (274)	total: 26.4s	remaining: 1